In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pennylane as qml
import umap
import math
import joblib
import pandas as pd
import os
from imblearn.under_sampling import TomekLinks
from collections import Counter

d:\QCNN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
labels_df = pd.read_csv("D:\\QCNN\\experiment-ii_labels.csv")
base_dir = "D:\\QCNN\\experiment-ii"

file_paths, unique_labels = [], []
X = []

# 1. Collect valid paths and labels
for _, row in labels_df.iterrows():
    subject = row["Subject"].strip()
    mat_type = row["Mat_Type"].strip()
    file_name = row["File_Name"].strip()
    label = row["Label"].strip()

    full_path = os.path.normpath(os.path.join(base_dir, subject, mat_type, file_name))
    if os.path.exists(full_path):
        file_paths.append(full_path)
        unique_labels.append(label)
    else:
        print("❌ Missing file:", os.path.abspath(full_path))

# 2. Load and flatten each valid pressure matrix only once
for path in file_paths:
    try:
        matrix = np.loadtxt(path)
        X.append(matrix.flatten())
    except Exception as e:
        print(f"Failed to load {path}: {e}")

# 3. Encode labels
label_mapping = {label: idx for idx, label in enumerate(sorted(set(unique_labels)))}
labels = [label_mapping[lbl] for lbl in unique_labels]

print("\n✅ Label Mapping:", label_mapping)
print("✅ Unique Encoded Labels:", np.unique(labels))

X = np.array(X)
Y = np.array(labels)



✅ Label Mapping: {'Left': 0, 'Right': 1, 'Supine': 2}
✅ Unique Encoded Labels: [0 1 2]


In [4]:
# Now fit and save
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

os.makedirs("preprocess", exist_ok=True)
joblib.dump(scaler, 'scaler8.pkl')
print("Scaler saved to scaler8.pkl")


Scaler saved to scaler8.pkl


In [5]:
# 2. Autoencoder for initial dimensionality reduction
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, bottleneck_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, bottleneck_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(bottleneck_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        return self.decoder(encoded)

    def encode(self, x):
        return self.encoder(x)

input_dim = X_scaled.shape[1]
bottleneck_dim = 50
autoencoder = AutoEncoder(input_dim, bottleneck_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)

In [6]:
# Train Autoencoder
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
for epoch in range(100):
    optimizer.zero_grad()
    output = autoencoder(X_tensor)
    loss = criterion(output, X_tensor)
    loss.backward()
    optimizer.step()

X_encoded = autoencoder.encode(X_tensor).detach().cpu().numpy()

In [7]:
# 3. UMAP for further dimensionality reduction (to match qubit count)
n_qubits = 11
umap_model = umap.UMAP(n_components=n_qubits, random_state=42)
X_umap = umap_model.fit_transform(X_encoded)

import joblib
joblib.dump(umap_model, "umap8_model.pkl")

d:\QCNN\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\QCNN\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


['umap8_model.pkl']

In [9]:
# 4. Handle class imbalance using SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_umap, Y)

# Print class distribution after SMOTE
label_counts = Counter(y_resampled)
print("Class distribution after SMOTE:")
for label, count in label_counts.items():
    print(f"Label {label}: {count} samples")

Class distribution after SMOTE:
Label 2: 271 samples
Label 1: 271 samples
Label 0: 271 samples


In [10]:
# 5. Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [11]:
# -----------------------------
# Configurable Parameters
# -----------------------------
NUM_QUBITS = 11
NUM_CLASSES = 3
NUM_Q_LAYERS = 2  # can increase if needed

In [12]:
# Quantum Convolution Layer
dev_conv = qml.device("default.qubit", wires=NUM_QUBITS)

# Quantum Convolution Layer
@qml.qnode(dev_conv, interface="torch", diff_method="backprop")
def QuantumConvLayer(inputs, weights):
    for i in range(NUM_QUBITS):
        qml.RY(inputs[i], wires=i)

    for pair_idx, i in enumerate(range(0, NUM_QUBITS - 1, 2)):
        w = weights[pair_idx]

        qml.U3(w[0], w[1], w[2], wires=i)
        qml.U3(w[3], w[4], w[5], wires=i + 1)

        qml.RY(w[6], wires=i)
        qml.RZ(w[7], wires=i + 1)
        qml.CNOT(wires=[i, i + 1])

        qml.RY(w[8], wires=i)
        qml.RY(w[9], wires=i + 1)
        qml.CNOT(wires=[i + 1, i])

        qml.U3(w[10], w[11], w[12], wires=i)
        qml.U3(w[13], w[14], w[15], wires=i + 1)

    # Convert the expectation values to PyTorch tensors (detached)
    return torch.stack([torch.tensor(qml.expval(qml.PauliZ(i)).real.item(), dtype=torch.float32) for i in range(NUM_QUBITS)]).detach()


In [13]:
# Quantum Pooling Layer
dev_pool = qml.device("default.qubit", wires=NUM_QUBITS)

# Quantum Pooling Layer
@qml.qnode(dev_pool, interface="torch", diff_method="backprop")
def QuantumPoolingCircuit(x):
    qml.AngleEmbedding(x, wires=range(NUM_QUBITS), rotation='Y')

    for i in range(0, NUM_QUBITS - 1, 2):
        qml.CRY(math.pi / 2, wires=[i, i + 1])
        qml.CZ(wires=[i, i + 1])

    for i in range(NUM_QUBITS):
        qml.Hadamard(wires=i)
        qml.S(wires=i)

    # Convert the expectation values to PyTorch tensors (detached)
    return torch.stack([torch.tensor(qml.expval(qml.PauliZ(i)).real.item(), dtype=torch.float32) for i in range(NUM_QUBITS)]).detach()


In [14]:
import torch.nn as nn
import torch.nn.functional as F

NUM_Q_LAYERS = 3     # or however deep your model is
NUM_CLASSES = 4      # update based on your classification task

class QCNN(nn.Module):
    def __init__(self, num_qubits=NUM_QUBITS, num_layers=NUM_Q_LAYERS, num_classes=NUM_CLASSES):
        super(QCNN, self).__init__()
        self.num_qubits = num_qubits
        self.num_layers = num_layers

        self.q_params = nn.Parameter(0.01 * torch.randn(num_layers, (num_qubits - 1) // 2, 16))

        # Classical layers
        self.fc1 = nn.Linear(num_qubits, 128)
        self.norm1 = nn.LayerNorm(128)
        self.fc2 = nn.Linear(128, 64)
        self.norm2 = nn.LayerNorm(64)
        self.fc3 = nn.Linear(64, 32)
        self.norm3 = nn.LayerNorm(32)
        self.fc4 = nn.Linear(32, 16)
        self.out = nn.Linear(16, num_classes)

    def encode_quantum(self, batch):
        quantum_outputs = []
        for sample in batch:
            x = sample
            for layer in range(self.num_layers):
                x = QuantumConvLayer(x, self.q_params[layer])
            pooled = QuantumPoolingLayer(x)
            quantum_outputs.append(pooled.detach())  # <-- Detach here
        return torch.stack(quantum_outputs)


    def forward(self, x, return_quantum_features=False):
        x = x.view(x.size(0), -1)
        quantum_features = self.encode_quantum(x).to(x.device)

        if return_quantum_features:
            return self.out(self.fc4(self.fc3(self.fc2(self.fc1(quantum_features))))), quantum_features

        x = F.relu(self.norm1(self.fc1(quantum_features)))
        x = F.dropout(x, p=0.3)
        x = F.relu(self.norm2(self.fc2(x)))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.norm3(self.fc3(x)))
        x = F.dropout(x, p=0.1)
        x = F.relu(self.fc4(x))
        return self.out(x)


In [17]:
# Set up model, optimizer, and loss function
model = QCNN()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)

# Assuming class_weights are computed based on  
loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights) 

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X_umap, Y, test_size=0.2, random_state=42)

# Convert the split data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

# Print the size of the datasets for confirmation
print(f"Training data shape: {X_train_tensor.shape}, Labels shape: {Y_train_tensor.shape}")
print(f"Testing data shape: {X_test_tensor.shape}, Labels shape: {Y_test_tensor.shape}")

Training data shape: torch.Size([372, 11]), Labels shape: torch.Size([372])
Testing data shape: torch.Size([93, 11]), Labels shape: torch.Size([93])


In [19]:
# Define batch size
batch_size = 32  # You can adjust this based on your system's memory and model size

# Create TensorDataset for training and testing sets
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

# Create DataLoader for training and testing with shuffling for training
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print the number of batches in each loader to confirm
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in test_loader: {len(test_loader)}")

Number of batches in train_loader: 12
Number of batches in test_loader: 3


In [20]:
# -------------------------------
# Training Configuration
# -------------------------------
epochs = 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_accuracies, train_losses = [], []
val_accuracies, val_losses = [], []

# -------------------------------
# Training Loop
# -------------------------------
for epoch in range(epochs):
    model.train()
    correct_train, total_train = 0, 0
    epoch_train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        try:
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
        except RuntimeError as e:
            print(f"Runtime error during training step: {e}")
            continue  # skip this batch if quantum circuit fails

        epoch_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = 100.0 * correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # -------------------------------
    # Validation Phase
    # -------------------------------
    model.eval()
    correct_val, total_val = 0, 0
    epoch_val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            try:
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
            except RuntimeError as e:
                print(f"Runtime error during validation step: {e}")
                continue

            epoch_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = epoch_val_loss / len(val_loader)
    val_accuracy = 100.0 * correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # -------------------------------
    # Epoch Summary
    # -------------------------------
    print(f"Epoch [{epoch+1}/{epochs}] → "
          f"Train Acc: {train_accuracy:.2f}% | Train Loss: {avg_train_loss:.4f} || "
          f"Val Acc: {val_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f}")


AttributeError: 'ExpectationMP' object has no attribute 'real'

In [ ]:
# Save the trained model
model_save_path = 'qcnn8_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(12, 6))
plt.plot(range(epochs), train_accuracies, label='Train Accuracy')
plt.plot(range(epochs), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(range(epochs), train_losses, label='Train Loss')
plt.plot(range(epochs), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
model.eval()

# Get the predictions for the test data
with torch.no_grad():
    # Convert test data to tensor
    test_data = torch.tensor(X_test, dtype=torch.float32).to(device)
    test_labels = torch.tensor(Y_test, dtype=torch.long).to(device)
    
    # Get the model's output (predictions)
    outputs = model(test_data)
    
    # Get the predicted class labels (highest probability)
    _, predicted_labels = torch.max(outputs, 1)

# Convert predictions and true labels to CPU for sklearn functions
predicted_labels = predicted_labels.cpu().numpy()
true_labels = test_labels.cpu().numpy()

# Confusion Matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix using seaborn
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", 
            xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
class_report = classification_report(true_labels, predicted_labels, target_names=label_mapping.keys())
print("Classification Report:")
print(class_report)